# How to Ride the Shopping Cart

## Abstract

Let’s face it: grocery shopping is not a skill most of us intentionally invested effort in pursuing. However, people more than often find themselves in situations where they return home with two bags of groceries and realize that they are missing the only item they initially were out for. Sometimes, not having a clear idea about our inventory state, we end up being tempted into the all sorts of treats that are over and above our necessities. Or we buy products to showcase our cooking abilities for the family dinner but somehow, we end up with the fridge full of everything, hoping to roll up our sleeves tomorrow. We are often surprised by the number of items we throw away because the expiration date passed two months ago! However, that could change...

## Main Research Question

In this notebook we try to address the following research question:

- Interplay between income and expenses?

### Sub-Questions

In particular, we are interested in the following sub-questions:

- How do households choose to organize their limited annual income according to their shopping expenses?

- Can we infer different household types based on the relation between their income and transaction statistics?

- Are some demographic properties of the household's indicators of this relation as well? That is, can we infer common family values indirectly from this data? 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout, Dropdown
from IPython.display import display
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS as colors

sns.set(style='whitegrid')

np.set_printoptions(suppress=True)

In [ ]:
DATA_DIR = 'data/'

CAMPAIGN_DESC = 'campaign_desc.csv'
CAMPAIGN_TABLE = 'campaign_table.csv'
CAUSAL_DATA = 'causal_data.csv'
COUPON = 'coupon.csv'
COUPON_REDEMPT = 'coupon_redempt.csv'
HH_DEMOGRAPHIC = 'hh_demographic.csv'
PRODUCT = 'product.csv'
TRANSACTION_DATA = 'transaction_data.csv'

## Part 1: Introduction

There are several factors that contribute to how people spend their annual income:

1. Income category
    - In our dataset we have several categories ranging from below **15,000** to over **250,000 USD p.a.**
2. Shopping behavior
    - It mainly depends on the day of the week
2. Campaigns play a major role
    - Some households take active participation in campaigns which significantly lowers the expenses
3. Products vary in price
    - Some products are much more expensive than others, nonetheless their purchase is unavoidable
4. Households preferences
    - Households with lower income tend to buy low-budget goods, as opposed to households with higher income that buy high-end goods (like organic food)
5. Demographics

To begin answering our question, we first perform data exploration on the **Dunnhumby** dataset to get initial insight for the task.

## Part 2: Preprocessing

### Loading the Data

For this research question, we will mainly work with the `hh_demographic` and `transaction_data` datasets.

In [ ]:
# Load the data from hh_demographic.csv
df_hh_demographic = pd.read_csv(DATA_DIR + HH_DEMOGRAPHIC, index_col='household_key')

print('Number of households:', len(df_hh_demographic))

df_hh_demographic.head()

In [ ]:
# Load the data from transaction_data.csv
df_transaction_data = pd.read_csv(DATA_DIR + TRANSACTION_DATA)

print('Number of transactions:', len(df_transaction_data))

df_transaction_data.head()

We observe that the `transaction_data` dataset contains a large number (circa 2.5 mil) of transactions. On the other hand, the `hh_demographic` dataset is much smaller in size (801).

Furthermore, there are transactions for which demographic data is not present. The reason behind the missing demographic information are privacy issues (as stated in the documentation).

### Handling Missing and Inconsistent Data

In [ ]:
print('Columns with missing values in household dataset:',
      np.any(df_hh_demographic.isna().sum()))

print('Columns with missing values in transactions dataset:',
      np.any(df_transaction_data.isna().sum()))

There are none missing values in both datasets.  

However, in the `transaction_data` dataset we observed that some entries contain 0 for `SALES_VALUE` (the total amount received by the retailer) and for `QUANTITY` (the number of products purchased). There may be a valid reason why the data had been recorded this way, however we chose to avoid any assumptions and discard these transactions, as it is only a small percentage of the whole dataset. 

In [ ]:
# Store the valid transactions in a new dataframe
c = (df_transaction_data['SALES_VALUE'] != 0) & \
    (df_transaction_data['QUANTITY'] != 0)

df_transaction_data_cleaned = df_transaction_data[c]

print('Number of transactions:', len(df_transaction_data_cleaned))

Furthermore, following the previous discussion about missing demographic information, we filter the data:

In [ ]:
# Keep only the transactions of the households for which we have demographic data
c = df_transaction_data_cleaned['household_key'].isin(df_hh_demographic.index)

df_transaction_data_cleaned = df_transaction_data_cleaned[c]

print('Number of transactions:', len(df_transaction_data_cleaned))

The `transaction_data` now is half its size before.

**Note**: Although, technically there are none missing values in both datasets, still, the columns: `MARITAL_STATUS_CODE`, `HOMEOWNER_DESC`, and `KID_CATEGORY_DESC` in the `hh_demographic` dataset, take values `U`, `Unknown`, and `None/Unknown` respectively which are unknown values. However, since the columns are categorical, we treat this values as yet another category with hope that later on we can obtain some useful insights.

### Feature Expansion

Using the formulas given in the dataset documentation, we can compute the values most useful for our analysis.

In [ ]:
sales_value = df_transaction_data_cleaned['SALES_VALUE']
coupon_desc = df_transaction_data_cleaned['COUPON_DISC']
retail_disc = df_transaction_data_cleaned['RETAIL_DISC']
coupon_match_disc = df_transaction_data_cleaned['COUPON_MATCH_DISC']
quantity = df_transaction_data_cleaned['QUANTITY']

After reading the dataset documentation, we noted that the price noted in the `SALES_VALUE` column is only the amount of money the retailer i.e. the store received from the customer. However, as we wish to know how much the *customer* spent and how much did the product they bought actually cost, we have to infer this information from the data. Fortunately, in the dataset documents formulas are provided to calculate these quantities.

In the `CUSTOMER_PAID` column we store the amount the customer had to pay, which is the `SALES_VALUE` reduced by the discount amount provided if the customer possessed a coupon from the product manufacturer, stored in the `COUPON_DISC` column.

On the other hand, in the `PRODUCT_PRICE` column we store the actual price of the product, which can be calculated by first re-adding back the discounts provided by the retailer's coupon and loyalty card possession from the manufacturer (the `COUPON_MATCH_DISC` and `RETAIL_DISC` columns) and then just dividing by the product `QUANTITY`. 

In [ ]:
# Use the provided formulas to calculate the new features and store them as new columns
df_transaction_data_cleaned['CUSTOMER_PAID'] = sales_value + coupon_desc
df_transaction_data_cleaned['PRODUCT_PRICE'] = (sales_value - (retail_disc + coupon_match_disc)) / quantity

### Type Conversion

In [ ]:
income_desc_categories = [
    'Under 15K',
    '15-24K',
    '25-34K',
    '35-49K',
    '50-74K',
    '75-99K',
    '100-124K',
    '125-149K',
    '150-174K',
    '175-199K',
    '200-249K',
    '250K+'
]

# Convert the INCOME_DESC feature column from string type to ordinal (ordered categorical)
df_hh_demographic['INCOME_DESC'] = pd.Categorical(df_hh_demographic['INCOME_DESC'],
                                                  categories=income_desc_categories,
                                                  ordered=True)

We could also typecast the other columns, but for now only `INCOME_DESC` is required to perform the analysis.

### What's in the data?

All the columns in the `hh_demographic` are categorical. On the other hand, the columns in `transaction_data` are numerical. More information regarding the value ranges is given in the readme file.

Lets analyse the two derived columns in `transaction_data`: `CUSTOMER_PAID` and `PRODUCT_PRICE`. All the necessary information about a transaction is captured by these two columns. The other columns have data about the basket, time, product, and the retail store, which are not relevant at this moment.

In [ ]:
df_transaction_data_cleaned[['CUSTOMER_PAID','PRODUCT_PRICE']] \
    .describe() \
    .loc[['min', 'max']]

We see that, in certain cases, the buyer didn't have to pay a penny as opposed to cases where the maximum bill amount was as high as \\$840. Also, there is a clear difference between the cheapest and most expensive product bought from the retail store.

Why is the amount paid negative though?

In [ ]:
# Retrieve the transactions with negative values for the amount paid by the customer
df_transaction_data_cleaned[df_transaction_data_cleaned['CUSTOMER_PAID'] < 0].head()

That's the effect of campaigns! During campaigns, that the stores and their customers participate in, coupons can be used to reduce the amount the customer needed to pay for a product. As these coupons usually have fixed discounts, if the discount awarded is higher that the product price we get negative values when calculating the amount payed by the customer. As this is not what really happened (the customer payed \\$0 instead of receiving money from the store), we clamp these negative values to 0 in order for our subsequent analyses to be correct.

In [ ]:
# For all of these transactions, clamp the customer payment amount to 0
df_transaction_data_cleaned.loc[df_transaction_data_cleaned['CUSTOMER_PAID'] < 0, 'CUSTOMER_PAID'] = 0

Let's plot the distributions of `CUSTOMER_PAID` and `PRODUCT_PRICE` (we will work with a sub-sample of 10K elements since the whole dataset contains over 1 million records and it fails to fit the distributions we assume they are following).

In [ ]:
def plot_distribution(df, r, column_name):
    '''
    Function to check the distribution of the derived columns
    
    df - dataframe with one column with values of the statistic
    r - right limit value of x-axis 
    column_name - name of the derived column
    '''

    xmin, xmax = 1e-6, r
    lnspc = np.linspace(xmin, xmax, len(df))
    m, s = stats.expon.fit(df)
    pdf_exponential = stats.expon.pdf(lnspc, scale=s)
    a, m, s = stats.powerlaw.fit(df)
    pdf_powerlaw = stats.powerlaw.pdf(lnspc, a, scale=s)

    fig = make_subplots(rows=1,
                        cols=2,
                        subplot_titles=('Linear scale', 'Logarithmic scale'))

    fig.add_histogram(x=df,
                      nbinsx=10,
                      hoverinfo='x+y',
                      marker_color=colors[0],
                      row=1,
                      col=1,
                      name = 'Lin. scale bins')
    fig.add_scatter(x=lnspc,
                    y=pdf_exponential * len(df),
                    mode='lines',
                    line_color=colors[3],
                    hoverinfo='x+y',
                    row=1,
                    col=1,
                    name = 'Exponential fit')
    fig.add_scatter(x=lnspc,
                    y=pdf_powerlaw * len(df),
                    mode='lines',
                    line_color=colors[2],
                    hoverinfo='x+y',
                    row=1,
                    col=1,
                    name = 'Power law fit')
    fig.update_xaxes(title_text='Product price (in $)',
                     range=[xmin, xmax],
                     row=1,
                     col=1,
                     gridcolor='rgb(230,230,230)')
    fig.update_yaxes(title_text='Number of examples', range=[0, 5000], row=1, col=1, gridcolor='rgb(230,230,230)')

    fig.add_histogram(x=df,
                      nbinsx=10,
                      hoverinfo='x',
                      marker_color=colors[0],
                      row=1,
                      col=2,
                      name = 'Log. scale bins')
    fig.add_scatter(x=lnspc,
                    y=pdf_exponential * len(df),
                    mode='lines',
                    line_color=colors[3],
                    hoverinfo='x+y',
                    row=1,
                    col=2,
                    name = 'Exponential fit')
    fig.add_scatter(x=lnspc,
                    y=pdf_powerlaw * len(df),
                    mode='lines',
                    line_color=colors[2],
                    hoverinfo='x+y',
                    row=1,
                    col=2,
                    name = 'Power law fit')
    fig.update_xaxes(title_text='Product price (in $)',
                     range=[xmin, xmax],
                     row=1,
                     col=2,
                     gridcolor='rgb(230,230,230)')
    fig.update_yaxes(title_text='Number of examples',
                     range=[0, np.log10(5000)],
                     type='log',
                     row=1,
                     col=2,
                     gridcolor='rgb(230,230,230)')

    fig.update_layout(showlegend=True,
                      title_text='Sample distribution of {}'.format("product price with discount" if column_name == "CUSTOMER_PAID" else "product price without discount"),
                      title_x = 0.5,
                      plot_bgcolor='rgb(255,255,255)')
    fig.show(renderer='iframe_connected')

In [ ]:
r = 20
df = df_transaction_data_cleaned['CUSTOMER_PAID'].sample(n=10000, random_state=1)
df = df[(df > 0) & (df < r)] # Remove the outliers

In [ ]:
plot_distribution(df, r, 'CUSTOMER_PAID')

In [ ]:
r = 20
df = df_transaction_data_cleaned['PRODUCT_PRICE'].sample(n=10000, random_state=1)
df = df[(df > 0) & (df < r)] # Remove the outliers

In [ ]:
plot_distribution(df, r, 'PRODUCT_PRICE')

Looks like we have an exponential distribution for both columns.

### Joining Datasets

We join the two datasets to obtain the transactions for each household. 

In [ ]:
df_households_transactions = df_transaction_data_cleaned.join(df_hh_demographic, on='household_key')

print('Total number of transactions for the selected households:',
      len(df_households_transactions))

## Part 3: Data Analysis

### Analysis of the Underlying Distributions

The following analysis can help us discover the shopping habits of the households, regardless of their annual income.

#### Household Income

We noted that one of the major factors in determining the household expenses is the household income category. This is represented by the `INCOME_DESC` column. It holds information about the total anual income for each household.

The main challenge in answering our question is the fact that this column is non numeric - each income value is represented as a category which represents an interval of income values. Looking on the bright side, there is meaningful ordering of the categories as they are intervals. 

In [ ]:
income_desc = df_hh_demographic['INCOME_DESC'].value_counts().sort_index()
plot_data = income_desc.reset_index().sort_values('index', ascending=False)
plot_data.columns=['Annual income (in $)', 'Number of households']
plot_data = plot_data.reindex(index = plot_data.index[::-1])
#plot_data = plot_data.rename(columns = {'Annual income (in $)': 'Annual income (in USD)'})

fig = px.bar(plot_data,
             y='Number of households',
             x='Annual income (in $)',
             color='Number of households',
             color_continuous_scale=px.colors.sequential.Viridis, #'blues',
             orientation='v')
fig.update_layout(title_text='Distribution of annual income',
                  plot_bgcolor='rgb(255,255,255)',
                  title_x = 0.5)
fig.update_yaxes(gridcolor='rgb(230,230,230)')
fig.show(renderer='iframe_connected')

While there is nothing wrong with this plot on first sight, careful analysis reveils a large problem with it: The bins are not equidistant. Therefore, it is biased towards bins of larger range. For example, the *15-24K* bin only spans a *9K* income range, while the *200-249K* bin spans a *49K* range. For a fair plot, we need to divide all the bins by their range. That way, we get an approximate *income distribution density* of the number of households per *1K* income range.

In [ ]:
# Define the bin ranges for income, list of values in the form [right bin range, left bin range]
bins = np.array([
    [15,0],
    [24,15],
    [34,25],
    [49,35],
    [74,50],
    [99,75],
    [124,100],
    [149,125],
    [174,150],
    [199,175],
    [249,200],
    [350,250]
])

# Calculate the bin sizes (lengths), to be used for normalization
diff = bins[:,0]-bins[:,1]

# Calculate the geometric means of the bins, to be used for fitting a log-normal distribution curve
geo_means = np.sqrt(bins[:,0]*bins[:,1])
geo_means[0] = np.mean([bins[0,0],bins[0,1]])

The `geo_means` variable is relevant, as it approximately represents the average value within every bin (assuming a log-normal distribution). The geometric mean between the lower and upper bound of a bin is more accurate than taking the average between the upper and lower bound. 

In [ ]:
# Calculate the income densities by dividing by the bin sizes
income_desc = df_hh_demographic['INCOME_DESC'].value_counts().sort_index()
income_desc_density = (income_desc / diff).values

In [ ]:
# Creating samples for the lognormal distribution
accuracy_param = 7 # This parameter is used to create an array which is sampled from the log-normal
samples = [[np.round(geo_mean), ] * int(accuracy_param * density)
           for geo_mean, density in zip(geo_means, income_desc_density)]
samples = [j for sub in samples for j in sub]

In [ ]:
from scipy.stats import lognorm
sigma, mu, alpha = lognorm.fit(np.array(samples)/accuracy_param)

In [ ]:
plot_data = (income_desc / diff).reset_index().sort_values('index', ascending=False)
plot_data.columns=['Annual income (in $)', 'Density of households']
plot_data = plot_data.reindex(index = plot_data.index[::-1])
#plot_data = plot_data.rename(columns = {'Annual income (in $)': 'Annual income (in USD)',
#                                        'Density of households': 'Number of households'})
plot_data = plot_data.rename(columns = {'Density of households': 'Number of households'})

fig = px.bar(plot_data,
             x='Annual income (in $)',
             y='Number of households',
             color='Number of households',
             color_continuous_scale=px.colors.sequential.Viridis,
             orientation='v')
fig.update_layout(title_text='Distribution of annual income (normalized on geometric mean of income range)',
                  plot_bgcolor = 'rgb(255, 255, 255)',
                  title_x = 0.5)
fig.update_yaxes(gridcolor='rgb(230,230,230)')
fig.show(renderer='iframe_connected')

Here we can see that the distribution looks like a log-normal distribution, which was to be expected given the Central Limit Theorem for positive values (income can only take values larger than zero, so the distribution has to be log-normal, instead of normal).

In [ ]:
import scipy.stats as ss

def plot_f(x_range, mu, sigma, alpha):
    '''
    Plots the f distribution function for a given x range, dfn and dfd
    If mu and sigma are not provided, standard f is plotted
    Passes any keyword arguments to matplotlib plot function
    '''
    x = x_range
    #y = ss.f.pdf(x, dfn, dfd, mu, sigma)
    #y = 1/((x-mu)*np.sqrt(2*np.pi*sigma**2))*np.exp(-np.log((x-mu)/alpha)**2/2/sigma**2)
    y = lognorm.pdf(x, s=sigma, loc=mu, scale=alpha)
    y_scaling = np.max(income_desc_density)/np.max(y)
    
    fig = make_subplots()
    fig.add_scatter(x=x*10, y=y*y_scaling, mode='lines', name='Log-normal fit')
    fig.add_scatter(x=geo_means, y=income_desc_density, mode='markers', name='Geometric means')
    fig.update_xaxes(title_text='Annual income (in $)',
                     range=[0, 300],
                     gridcolor = 'rgb(230, 230, 230)')
    fig.update_yaxes(title_text='Number of households',
                     gridcolor = 'rgb(230, 230, 230)')

    fig.update_layout(title_text='Log-normal distribution fit of annual income',
                      plot_bgcolor = 'rgb(255, 255, 255)',
                      title_x = 0.5)
    fig.show(renderer='iframe_connected')

x = np.linspace(0.001, 100, 500)
plot_f(x, mu, sigma, alpha)

#### Average Bill Amount

To measure the spending habits of the households, we compute the average amount of households' bills. Each household visit to a store is uniquely identified with a `BASKET_ID`. We do this in two steps:

1. We sum the derived `CUSTOMER_PAID` column across the transactions for each visit to get the total bill amount for the visit. <br>
2. We average across all visits of each household.

In [ ]:
df_households_basket_payment = df_households_transactions \
    .groupby(['household_key', 'BASKET_ID'])['CUSTOMER_PAID'] \
    .sum()

df_households_basket_payment_average = df_households_basket_payment.groupby(['household_key']).mean()

In [ ]:
from scipy.stats import iqr
def calculate_bin_size(data):
    '''Function to calculate the appropriate histogram bin size using the Freedman-Diaconis rule'''
    return 2 * iqr(data) / (len(data)**(1 / 3))

In [ ]:
hist_data = [list(df_households_basket_payment_average),]
group_labels = [None, ]
fig = ff.create_distplot(hist_data, group_labels, bin_size=calculate_bin_size(hist_data[0]), show_rug=False)
fig.data[0]['hoverinfo'] = 'x+y'
fig.data[1]['hoverinfo'] = 'x+y'
fig.update_layout(title_text='Distribution of mean bill amount',
                  xaxis_title='Mean bill amount (in $)',
                  yaxis_title='Density of households',
                  showlegend=False,
                  title_x = 0.5,
                  plot_bgcolor = 'rgb(255, 255, 255)')
fig.update_yaxes(gridcolor = 'rgb(230, 230, 230)')
fig.show(renderer='iframe_connected')

Looking at the histogram of this statistic, we identify it as a right-skewed Gaussian distribution, with a few outliers.

On average, people spend around \\$30 at a local retailer, which is kind of expected. However, there are several outliers and this is further investigated below.

#### Average Number of Products
To measure the spending habits of the households, we can also compute statistics based on the products. For instance, the simplest idea would be to just count the number of different products bought and average across all visits.

In [ ]:
df_household_basket_num_products = df_households_transactions \
                                   .groupby(['household_key', 'BASKET_ID'])['PRODUCT_ID'] \
                                   .count()

df_households_basket_num_products_average = df_household_basket_num_products.groupby('household_key').mean()

In [ ]:
hist_data = [list(df_households_basket_num_products_average),]
group_labels = [None, ]
fig = ff.create_distplot(hist_data, group_labels, bin_size=calculate_bin_size(hist_data[0]), show_rug=False)
fig.data[0]['hoverinfo'] = 'x+y'
fig.data[1]['hoverinfo'] = 'x+y'
fig.update_layout(title_text='Distribution of mean number of purchased products per visit',
                  xaxis_title='Mean number of products',
                  yaxis_title='Density of households',
                  showlegend=False,
                  title_x = 0.5,
                  plot_bgcolor = 'rgb(255, 255, 255)')
fig.update_yaxes(gridcolor = 'rgb(230, 230, 230)')
fig.show(renderer='iframe_connected')

Similarly as before, from the histogram we identify this statistic as having a right-skewed Gaussian distribution, with a few outliers.

On average, people buy around 10 products and it is more likely to observe a greater than average number of products bought during a shopping visit than lesser, which is expected - hence the longer right tail.

#### Average Median Product Price

Product prices are more reflective of the expenses that households have, rather than just counting the number of products.

From this perspective, we can calculate a different statistic quantifying the spending habits of each household. This time we use the derived `PRODUCT_PRICE` to calculate the median product price in each basket and then average across for each household.

We calculate this statistic because the median is a good indicator of the households spending habit during a trip. The average, lowest or greatest price of a product in the basket would not be relevant, as they are very sensitive to outliers while the median is a robust statistic.

The reason for the prevalence of outliers in the product prices, stem for the fact that they are the actual value of a single product and do not take into account the product quantity in the bill, unlike the `CUSTOMER_PAID` values.  

In [ ]:
df_households_basket_median_product_price = df_households_transactions \
    .groupby(['household_key', 'BASKET_ID'])['PRODUCT_PRICE'] \
    .median()

df_households_basket_median_product_price_average = df_households_basket_median_product_price.groupby('household_key').mean()

In [ ]:
hist_data = [list(df_households_basket_median_product_price_average),]
group_labels = [None, ]
fig = ff.create_distplot(hist_data, group_labels, bin_size=calculate_bin_size(hist_data[0]), show_rug=False)
fig.data[0]['hoverinfo'] = 'x+y'
fig.data[1]['hoverinfo'] = 'x+y'
fig.update_layout(title_text='Distribution of average median product price',
                  xaxis_title='Average median product price (in $)',
                  yaxis_title='Density of households',
                  showlegend=False,
                  title_x = 0.5,
                  plot_bgcolor = 'rgb(255, 255, 255)')
fig.update_yaxes(gridcolor = 'rgb(230, 230, 230)')
fig.show(renderer='iframe_connected')

We observe from the histogram that this feature has a bit more regular Gaussian distribution with a mean around \\$2\.50. However, there are also outliers in this statistic. This is depicted in detail in another study below.

### Statistics Comparison

#### Correlation

We are interested in whether these exist any correlation between these three statistics. Does the median product price determine the actual value on the bill?

Using a scatter plot, we would like to confirm (or deny) this.

In [ ]:
# Compute the Spearman correlation coefficient between the two statistics
spearman_corr_coef = df_households_basket_payment_average.corr(df_households_basket_median_product_price_average,
                                                               method='spearman')
print('Correlation coefficient:', spearman_corr_coef)

plot_data = pd.concat((df_households_basket_payment_average, df_households_basket_median_product_price_average), axis=1)
plot_data.columns = ['Average bill amount (in $)', 'Average median product price (in $)']
#plot_data = plot_data.rename(columns = {'Average bill amount (in $)': 'Average bill amount (in USD)',
#                                        'Average median product price (in $)': 'Average median product price (in USD)'})

fig = px.scatter(data_frame=plot_data,
                 x='Average bill amount (in $)',
                 y='Average median product price (in $)',
                 trendline='ols')
fig.update_layout(title_text='Average bill amount vs average median product price', #'Correlation between the two statistics',
                  title_x = 0.5,
                  plot_bgcolor = 'rgb(255, 255, 255)')
fig.update_xaxes(gridcolor = 'rgb(230, 230, 230)')
fig.update_yaxes(gridcolor = 'rgb(230, 230, 230)')
fig.show(renderer='iframe_connected')

Interestingly, we observe almost no correlation between these two statistics! This would mean that the representative price of the products in the basket is very unrelated to the actual amount payed at the register. As such, these two features measure two very different aspects of a household's spending habits and must be used together as a pair to describe a household.

In [ ]:
# Compute the Spearman correlation coefficient between the two statistics
spearman_corr_coef = df_households_basket_num_products_average.corr(df_households_basket_payment_average,
                                                               method='spearman')
print('Correlation coefficient:', spearman_corr_coef)

plot_data = pd.concat((df_households_basket_num_products_average, df_households_basket_payment_average), axis=1)
plot_data.columns = ['Average number of products', 'Average bill amount (in $)']
#plot_data = plot_data.rename(columns = {'Average bill amount (in $)': 'Average bill amount (in USD)'})

fig = px.scatter(plot_data,
                 x='Average number of products',
                 y='Average bill amount (in $)',
                 trendline='ols')
fig.update_layout(title_text='Average number of products vs average bill amount', #'Correlation between the two statistics',
                  title_x = 0.5,
                  plot_bgcolor = 'rgb(255, 255, 255)')
fig.update_xaxes(gridcolor = 'rgb(230, 230, 230)')
fig.update_yaxes(gridcolor = 'rgb(230, 230, 230)')
fig.show(renderer='iframe_connected')

Unlike the previous pair, the mean number of products and the average bill amounts seem to be almost perfectly correlated! This was expected however: as you buy more products the more will the bill amount increase. As this statistic is not a good descriptor anyway, we have proved that we can discard it as it does not bring new information.

### Analysis of the Joint Distribution 

Now that we have insight on how much money, on average, people spend on each visit to the store and how does the representative product price bought vary, we are interested to see does the household income play a crucial role in this. Our expectation is that it does, and that households with lower income spend less and buy cheaper products as opposed to households with higher income.

#### Household Income vs. Average Bill Amount

Using a boxplot we can view the distribution of the statistic across each household income category and perform a comparison very efficiently.

In [ ]:
# Combine the income values and average bill amount values for each household in a new dataframe
df = df_hh_demographic[['INCOME_DESC']].join(df_households_basket_payment_average)

In [ ]:
plot_data = df[['CUSTOMER_PAID', 'INCOME_DESC']]
plot_data.columns = ['Average bill amount (in $)', 'Annual income (in $)']
#plot_data = plot_data.rename(columns = {'Average bill amount (in $)': 'Average bill amount (in USD)',
#                                        'Annual income (in $)': 'Annual income (in USD)'})

fig = px.box(data_frame=plot_data,
             x='Average bill amount (in $)',
             y='Annual income (in $)',
             orientation='h')
fig.update_layout(
    title_text=
    'Distribution of average bill amount per annual income categories',
    yaxis_categoryorder='array',
    yaxis_categoryarray=income_desc_categories,
    yaxis_autorange='reversed',
    title_x = 0.5,
    plot_bgcolor = 'rgb(255, 255, 255)')
fig.update_xaxes(gridcolor = 'rgb(230, 230, 230)')
fig.show(renderer='iframe_connected')

To our surprise, extreme spending are mostly observed for medium and lower income households.

Using a 2D histogram i.e. heatmap we can compare the relationship between income and the statistic in more detail by viewing their joint distribution.

In [ ]:
# Convert the income column from ordinal to string type (needed for plotting compatibility)
mapping = dict(zip(df['INCOME_DESC'], df['INCOME_DESC'].cat.codes))
df['INCOME_DESC'] = df['INCOME_DESC'] \
    .astype(str) \
    .map(mapping)

In [ ]:
# Use Kernel Density Estimation to estimate the joint distribution from the 2D histogram
g = sns.jointplot(y='INCOME_DESC', x='CUSTOMER_PAID', data=df, kind='kde', height=10)
g.set_axis_labels(
    'Average bill amount (in $)',
    'Annual income (in $)'
)
g.ax_joint.set_yticklabels(income_desc_categories);

We can observe that the majority of the households are concentrated at the medium income values of 35-74K and lower average expenses of about \\$20 - \\$40.

#### Household Income vs. Average Median Product Price

Similarly, to begin the comparison between household income and median product prices we first merge the two features of each household.

In [ ]:
# Combine the income values and average max product price values for each household in a new dataframe
df = df_hh_demographic[['INCOME_DESC']].join(df_households_basket_median_product_price_average)

In [ ]:
plot_data = df[['PRODUCT_PRICE', 'INCOME_DESC']]
plot_data.columns = ['Average median product price (in $)', 'Annual income (in $)']

fig = px.box(data_frame=plot_data,
             x='Average median product price (in $)',
             y='Annual income (in $)',
             orientation='h')
fig.update_layout(
    title_text='Distribution of average median product price per annual income categories',
    yaxis_categoryorder='array',
    yaxis_categoryarray=income_desc_categories,
    yaxis_autorange='reversed',
    title_x = 0.5,
    plot_bgcolor = 'rgb(255, 255, 255)')
fig.update_xaxes(gridcolor = 'rgb(230, 230, 230)')
fig.show(renderer='iframe_connected')

The phenomena related to the extreme values is present here as well. Additionally, an interesting difference is the generally smaller value ranges for all of the income categories, capturing the fact that the median has not considered many outlier products with very low or very high prices.

In [ ]:
# Convert the income column from ordinal to string type (needed for plotting compatibility)
mapping = dict(zip(df['INCOME_DESC'], df['INCOME_DESC'].cat.codes))

df['INCOME_DESC'] = df['INCOME_DESC'].astype(str).map(mapping)

In [ ]:
# Use Kernel Density Estimation to estimate the joint distribution from the 2D histogram
g = sns.jointplot(y='INCOME_DESC', x='PRODUCT_PRICE', data=df, kind='kde', height=10)
g.set_axis_labels(
    'Average median product price (in $)',
    'Annual income (in $)'
)
g.ax_joint.set_yticklabels(income_desc_categories);

Analyzing the joint distribution, we observe that it is similar in form to the one for the previous statistic, although more narrow in the x direction - due to the smaller value ranges of the median as also previously seen with the boxplot. We observe that the most common occurrence is a household with income in the range 50-74K to have an average median product price of about 2\.5$.

### Trends

#### Coupon usage

We noticed that campaigns play a major role in how people spend their money. In deriving the true amount of money that households spend on every visit to the store, we took into account the coupon value. But what is the total amount used and does it vary with the income category?

In [ ]:
# For each household income category calculate the total coupon discount amount used
# and normalize it by dividing it with the number of households in that category
plot_data = df_households_transactions.groupby(['INCOME_DESC'], observed=True)['COUPON_DISC'] \
    .sum() \
    .div(df_hh_demographic['INCOME_DESC'].value_counts()) \
    .reset_index()
plot_data.columns = ['Annual income (in $)', 'Average total coupon discount amount (in $)']
plot_data['Average total coupon discount amount (in $)'] = abs(plot_data['Average total coupon discount amount (in $)'])
plot_data = plot_data.rename(columns = {'Average total coupon discount amount (in $)': 'Mean total discount'})

fig = px.bar(data_frame=plot_data,
             x='Annual income (in $)',
             y='Mean total discount',
             orientation='v',
             color = 'Mean total discount',
             color_continuous_scale=px.colors.sequential.Viridis)
#fig.data[0]['marker']['color'] *= -1
fig.update_layout(
    title_text='Distribution of coupon usage',
    yaxis_categoryorder='array',
    yaxis_categoryarray=income_desc_categories,
    title_x = 0.5,
    plot_bgcolor = 'rgb(255, 255, 255)')
    #yaxis_autorange='reversed')
fig.update_yaxes(gridcolor = 'rgb(230, 230, 230)')
fig.show(renderer='iframe_connected')

Coupon value is negative, since it gives a discount to the price. On this bar plot, we see a somewhat unexpected trend. People with medium to upper medium income use more coupons as opposed to the people with lower income. Interesting sight is that people with high income are also heavy users of coupons.

#### Daily Expenses

Interesting information can be found by looking into the day of the week when purchases have been made. While we don't know the exact date (and thereby day of the week) from when the data was recorded, it is enough to group the transactions by the *day number* module *7*, in order to plot the transactions throughout an average week. 

In [ ]:
df = df_households_transactions.copy()

# Map the integer DAY values(1-711) to weekdays (0-6) using modulo 7
df['DAY'] = df['DAY'].apply(lambda d: d % 7)

# Calculate average customer payments for each weekday
df = df.groupby(['DAY'])['CUSTOMER_PAID'] \
    .sum() \
    .reset_index()

df = df['CUSTOMER_PAID'] / len(df_hh_demographic)

In [ ]:
plot_data = df.reset_index()
plot_data.columns = ['Day', 'Total bill amount (in $)']

days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
plot_data['Day'] = plot_data['Day'].apply(lambda x: days_of_week[x])
plot_data = plot_data.rename(columns = {'Day': 'Day of week'})
plot_data = plot_data.rename(columns = {'Total bill amount (in $)': 'Total bill amount'})

fig = px.bar(plot_data,
             x='Day of week',
             y='Total bill amount',
             color='Total bill amount',
             color_continuous_scale=px.colors.sequential.Viridis)
fig.update_layout(title_text="Distribution of bill amount per day of week",
                  title_x = 0.5,
                  plot_bgcolor = 'rgb(255, 255, 255)')
fig.update_yaxes(gridcolor = 'rgb(230, 230, 230)')
fig.show(renderer='iframe_connected')

While we cannot be certain about which day this weeks starts with, it is to be assumed that that the two days with a larger amount of transactions are either Friday and Saturday or Saturday and Sunday. If the data is from the US, it is likely that most purchases have been made on Sunday, as people tend to have more time for shopping, and the stores are usually open on Sunday. We contacted the company which provides this dataset for more information about such details, and are currently waiting for a response. 

#### Mapping Between Expenses and Product Categories

Here we are interested to find out how household expenditure depends on the goods and/or services and how does it vary with household income. For this analysis, we will load the `product.csv` dataset. It contains information about a particular product associated with a transaction.

In [ ]:
# Load the data from products.csv
df_product = pd.read_csv(DATA_DIR + PRODUCT, index_col='PRODUCT_ID')

print('Number of products:', len(df_product))

df_product.head()

In [ ]:
print('Columns with missing values in products dataset:',
      np.any(df_product.isna().sum()))

Although, there are no missing values, the `CURR_SIZE_OF_PRODUCT` column has empty string values. As this is not crutial for our analysis, we are not going to further address this issue. 

We join the household transactions dataset with the products dataset using the `PRODUCT_ID` column.

In [ ]:
df_transactions_products = \
    df_households_transactions[['PRODUCT_ID', 'QUANTITY', 'CUSTOMER_PAID', 'INCOME_DESC']] \
    .join(df_product[['COMMODITY_DESC', 'SUB_COMMODITY_DESC']], on='PRODUCT_ID')

df_transactions_products.head()

The following dataset is just an aggregate over `INCOME_DESC` and `COMMODITY_DESC`. It is usefull for our analysis that follow.

In [ ]:
df = df_transactions_products \
    .groupby(['INCOME_DESC', 'SUB_COMMODITY_DESC'], observed=True)['CUSTOMER_PAID'] \
    .sum() \
    .dropna() \
    .to_frame()

df.head()

#### Most Purchased Product

We are interested to see on which product does the households spend most of their money on. This is also dependent on the income category.

In [ ]:
# Calculate most purchased product as argmax of total customer payments for each income category
df.unstack().idxmax(axis=1)

It looks like they spend most of their anual income on gasoline, which is expected taking into account the demographics data of the households. But lets see the distribution.

In [ ]:
# Calculate the total money spent od unleaded gasoline for each income category
# and normalize with the number of households in each category
stats = df.unstack()['CUSTOMER_PAID']['GASOLINE-REG UNLEADED'] \
    .div(df_hh_demographic['INCOME_DESC'].value_counts().values) \
    .values

stats

In [ ]:
# Construct data into the form needed for polar coordinate plotting
# Discrete angle values represent income categories, radius represents normalized gasoline purchases
angles = np.linspace(0, 360, len(income_desc_categories), endpoint=False)
r = np.concatenate((stats, [stats[0]]))
theta = np.concatenate((angles, [angles[0]]))

plot_data = pd.concat(
    (pd.Series(r, name='Normalized gasoline purchases (in $)'),
     pd.Series(income_desc_categories + [income_desc_categories[0],],
               name='Annual income (in $)')),
    axis=1)

fig = px.scatter_polar(plot_data,
                    r='Normalized gasoline purchases (in $)',
                    theta='Annual income (in $)',
                    start_angle=180,
                    direction='clockwise',
                    title='Most purchased product category per income')

fig.update_layout(title_text='Most purchased product category per income',
                  title_x = 0.5,
                  plot_bgcolor = 'rgb(255, 255, 255)')

fig.update_traces(fill='toself', hoveron='points', mode='markers+lines')
fig.show(renderer='iframe_connected')

The distribution is highly concentrated around the middle to upper middle income categories. The reason is that households with lower income can't afford owning a vehicle, however the reason for the upper income is somewhat unclear. It could be that they use other means of transportation required by their job.

Setting gasoline aside, we are also interest in how the expenses are distributed in the other product categories.

In [ ]:
# Temporarily remove all purchase data about this gasoline category
dff = df[df.index.get_level_values('SUB_COMMODITY_DESC') != 'GASOLINE-REG UNLEADED']

dff.head()

For this, we will use a sankey plot because it is an excellent way to visualize flow of information between two subsets. From it, we will have a clear idea about how the expenses from income groups map to product categories.

We will use top 200 products ordered by `QUANTITY` plot the flow diagram between the income and the product category. We can easily visualize how the quantity of the purchased products is distributed and later search for patterns in the expenditure.

In [ ]:
# For each income category find the top 15 purchased product categories based on customer spending
dff = dff \
    .reset_index() \
    .groupby(['INCOME_DESC'], observed=True)['SUB_COMMODITY_DESC', 'CUSTOMER_PAID'] \
    .apply(lambda df: df.sort_values(by='CUSTOMER_PAID', ascending=False).head(15)) \
    .reset_index(level=0)

dff

In [ ]:
# Aggregate customer payment data first on income categories and then with product categories
# Construct data into form required for the Sankey plot
data = []

palette = sns.color_palette('cubehelix', len(dff) + dff['INCOME_DESC'].nunique())
colours = palette.as_hex()

level1 = dff[['INCOME_DESC', 'CUSTOMER_PAID']] \
    .groupby('INCOME_DESC', observed=True) \
    .agg('sum')

c_count = 0

for i, r in level1.reset_index().iterrows():
    data.append({
        'source': 'Expenditure',
        'target': r['INCOME_DESC'],
        'value': r['CUSTOMER_PAID'],
        'color': colours[c_count]
    })
    
    item_dff = dff[dff['INCOME_DESC'] == r['INCOME_DESC']] \
        [['INCOME_DESC', 'SUB_COMMODITY_DESC', 'CUSTOMER_PAID']] \
        .groupby(['INCOME_DESC', 'SUB_COMMODITY_DESC'], observed=True) \
        .agg('sum') \
        .reset_index()
    
    for item_i, item_r in item_dff.iterrows():
        
        c_count += 1

        data.append({
            'source': item_r['INCOME_DESC'],
            'target': item_r['SUB_COMMODITY_DESC'],
            'value': item_r['CUSTOMER_PAID'],
            'color': colours[c_count]
        })

In [ ]:
# Load the sankey plot instead!
#
# w = SankeyWidget(layout=Layout(width='1600', height='3000'),
#                  links=data,
#                  margins=dict(top=0, bottom=0, left=100, right=150))
#
# w.save_png('sankey.png')
#
# w

<img src='sankey.png' />

In order to analyze this sankey diagram further, we call on a principle from microeconomics, called **Engel curves**.

#### Engel curves

One particular way of determining how household expenditure, on a particular good or service, varies with household income is visualizing Engel curves. Engel curves on the x-axis have the income category and on the y-axis the quantity of the measured product. We can infer some type of goods using these curves.

In [ ]:
# Calculate total quantity of each product category purchased, for each income category 
df = df_transactions_products \
    .groupby(['INCOME_DESC', 'COMMODITY_DESC'], observed=True)['QUANTITY'] \
    .sum() \
    .dropna() \
    .reset_index(level=1)

For a start, here are the 10 most purchased items:

In [ ]:
df.groupby('COMMODITY_DESC')['COMMODITY_DESC','QUANTITY'].sum().sort_values('QUANTITY',ascending=False).head(10)

In [ ]:
# commodity_desc_list = df['COMMODITY_DESC'].unique()
commodity_desc_list = [
    'MARGARINES', 'FROZEN', 'PREPARED FOOD', 'PNT BTR/JELLY/JAMS',
    'SPICES & EXTRACTS', 'IMPORTED WINE',
    'FEMININE HYGIENE', 'FITNESS&DIET'
]
plot_data = pd.DataFrame()
for commodity_desc in commodity_desc_list:
    df_commodity = df[df['COMMODITY_DESC'] == commodity_desc] \
        .reindex(income_desc_categories, fill_value=0)
    df_commodity['COMMODITY_DESC'] = commodity_desc
    # Calculate the distribution of the average purchased quantity of the product category
    df_commodity['QUANTITY'] = df_commodity['QUANTITY'] \
        .div(df_hh_demographic['INCOME_DESC'].value_counts().values)
    plot_data = pd.concat((plot_data, df_commodity.reset_index()))

# Remove the first and last income category because
# they contain all the outliers
plot_data = plot_data[~plot_data['INCOME_DESC'].isin(['Under 15K', '250K+'])]
plot_data.columns = ['Annual income (in $)', 'Product category', 'Quantity']
fig = px.line(plot_data, x='Annual income (in $)', y='Quantity', color='Product category', line_shape='spline')
fig.update_traces(mode='markers+lines')

menu_buttons = [{'label':'ALL', 'method':'update', 'args':[{'visible':[True,] * len(commodity_desc_list)}]}]
for i, commodity_desc in enumerate(commodity_desc_list):
    button_arg_visible = [False, ] * len(commodity_desc_list)
    button_arg_visible[i] = True
    button = {'label':commodity_desc, 'method':'update', 'args':[{'visible':button_arg_visible}, ]}
    menu_buttons.append(button)
    
dropdown_menu = go.layout.Updatemenu(active=0, buttons=menu_buttons, showactive=True, xanchor="left", x=0, yanchor="top", y=1.125) #x=0.8
fig.update_layout(title_text = 'Engel curves',
                  updatemenus = [dropdown_menu, ],
                  title_x = 0.5,
                  plot_bgcolor = 'rgb(255, 255, 255)',
                  showlegend = False)
fig.update_xaxes(gridcolor = 'rgb(230, 230, 230)')
fig.update_yaxes(gridcolor = 'rgb(230, 230, 230)')
fig.show(renderer='iframe_connected')

Here we showcase an example of inferior goods. Margarine is an inferior good since its demand drops when people's incomes rise. This occurs when a good has more costly substitutes (butter) that see an increase in demand as incomes and the economy improve. There are other examples in our dataset (ex. frozen food, prepared food, peanut butter and jelly).

When looking at the above plots, one might be surprised that for all four products, households with low incomes, and households with extremely high income tend to purchase fewer of them. For households with low income, this is no surprise. But what about high-income households? It is likely that individuals of high incomes often dine out, as this saves time. That way, they don't have to purchase large quantities of any products in supermarkets. 

### Demographic Analysis of Household Groups

In order to understand how family values influence the balance between household's expenses and income, we will analyze the demographic properties across 4 groups of households.

In [ ]:
# Median values for income and expenses
# We will use the average bill amount statistic to represent expenses
AVG_INCOME = '50-74K'
AVG_EXPENSES = df_households_basket_payment_average.median()

In [ ]:
# Split the household demographic dataset into 4 subsets based on income and expenses
df = df_hh_demographic.join(df_households_basket_payment_average.to_frame().rename(columns={'CUSTOMER_PAID': 'EXPENSES'}))

df_low_income_low_expenses = df[(df['INCOME_DESC'] <= AVG_INCOME) & (df['EXPENSES'] <= AVG_EXPENSES)]
df_low_income_high_expenses = df[(df['INCOME_DESC'] <= AVG_INCOME) & (df['EXPENSES'] > AVG_EXPENSES)]
df_high_income_low_expenses = df[(df['INCOME_DESC'] > AVG_INCOME) & (df['EXPENSES'] <= AVG_EXPENSES)]
df_high_income_high_expenses = df[(df['INCOME_DESC'] > AVG_INCOME) & (df['EXPENSES'] > AVG_EXPENSES)]

In [ ]:
print('Percent of households with low income and low expenses:', round(len(df_low_income_low_expenses) / len(df_hh_demographic), 4))
print('Percent of households with low income and high expenses:', round(len(df_low_income_high_expenses) / len(df_hh_demographic), 4))
print('Percent of households with high income and low expenses:', round(len(df_high_income_low_expenses) / len(df_hh_demographic), 4))
print('Percent of households with high income and high expenses:', round(len(df_high_income_high_expenses) / len(df_hh_demographic), 4))

These groups were generated using the average income and average expenses. We split the households into: below (or above) average income and below (or above) average expenses.

In [ ]:
demographic_features = df_hh_demographic.columns.drop('INCOME_DESC')

demographic_features_categories = dict.fromkeys(demographic_features)
demographic_features_categories['AGE_DESC'] = ['19-24', '25-34', '35-44', '45-54', '55-64', '65+']
demographic_features_categories['MARITAL_STATUS_CODE'] = ['A', 'B', 'U']
demographic_features_categories['HOMEOWNER_DESC'] = ['Homeowner', 'Probable Owner', 'Renter', 'Probable Renter', 'Unknown']
demographic_features_categories['HH_COMP_DESC'] = ['2 Adults Kids', '1 Adult Kids', '2 Adults No Kids', 'Single Female', 'Single Male', 'Unknown']
demographic_features_categories['HOUSEHOLD_SIZE_DESC'] = ['1', '2', '3', '4', '5+']
demographic_features_categories['KID_CATEGORY_DESC']= ['None/Unknown', '1', '2', '3+']

demographic_features_clean_names = {'AGE_DESC': 'Age',
                                    'MARITAL_STATUS_CODE': 'Marital Status',
                                    'HOMEOWNER_DESC': 'Homeowner Type',
                                    'HH_COMP_DESC':'Household Composition',
                                    'HOUSEHOLD_SIZE_DESC':'Household Size',
                                    'KID_CATEGORY_DESC':'Number of children'}



plot_data = dict.fromkeys(demographic_features)
rename_data = {
    'AGE_DESC': {
        '19-24': '19-24',
        '25-34': '25-34',
        '35-44': '35-44',
        '45-54': '45-54',
        '55-64': '55-64',
        '65+'  : '65+'
    },
    
    'MARITAL_STATUS_CODE': {
        'A': 'Married',
        'B': 'Single',
        'U': 'Unknown'
    },
    
    'HOMEOWNER_DESC': {
        'Homeowner' : 'Homeowner',
        'Probable Owner': 'Portable owner',
        'Renter': 'Renter', 
        'Probable Renter': 'Probable renter',
        'Unknown': 'Unknown'
    },
    
    'HH_COMP_DESC': {
        '2 Adults Kids': '2 adults with kids', 
        '1 Adult Kids': '1 adult with kids',
        '2 Adults No Kids': '2 adults without kids',
        'Single Female': 'Single female',
        'Single Male': 'Single male',
        'Unknown': 'Unknown'
    },
    
    'HOUSEHOLD_SIZE_DESC': {
        '1': '1',
        '2': '2',
        '3': '3',
        '4': '4',
        '5+': '5+'
    },
    
    'KID_CATEGORY_DESC': {
        'None/Unknown': 'None / Unknown',
        '1': '1',
        '2': '2',
        '3+': '3+'
    }
}

for demographic_feature in demographic_features:
    df = pd.concat([
        df_low_income_low_expenses[demographic_feature].value_counts().rename('Low/Low') / len(df_low_income_low_expenses),
        df_low_income_high_expenses[demographic_feature].value_counts().rename('Low/High') / len(df_low_income_high_expenses),
        df_high_income_low_expenses[demographic_feature].value_counts().rename('High/Low') / len(df_high_income_low_expenses),
        df_high_income_high_expenses[demographic_feature].value_counts().rename('High/High') / len(df_high_income_high_expenses)
    ], axis=1, sort=False)
    df.index = demographic_features_categories[demographic_feature]
    df = df.unstack().reset_index()
    df.columns = ['Income vs Expenses', demographic_feature, 'Percentage of households']
    plot_data[demographic_feature] = df

def plot_demographic(demographic_feature):
    data = plot_data[demographic_feature].copy()
    data[demographic_feature] = data[demographic_feature].apply(lambda x: rename_data[demographic_feature][x])
    
    fig = px.bar(data_frame=data,
             y='Percentage of households',
             color='Income vs Expenses',
             barmode='group')
    fig.update_layout(xaxis_title=demographic_features_clean_names[demographic_feature],
                      xaxis_tickmode='array',
                      xaxis_tickvals=list(range(len(demographic_features_categories))),
                      xaxis_ticktext=list(map(lambda x: rename_data[demographic_feature][x], demographic_features_categories[demographic_feature])),
                      title_x = 0.5,
                      plot_bgcolor = 'rgb(255, 255, 255)')
    fig.update_yaxes(gridcolor = 'rgb(230, 230, 230)')
    fig.show(renderer='iframe_connected')

#### Age

In [ ]:
plot_demographic('AGE_DESC')

From the age distribution of the household groups, we can make a few interesting observations. Among the households with the youngest members there is not a lot of variety in the income-expenses balance. Among the households with members of younger working ages high expenses seem to dominate. As we move to the households with older members lower expenses are more prevalent.

#### Marital Status

In [ ]:
plot_demographic('MARITAL_STATUS_CODE')

For clarification, according to the dataset documentation the marital status codes `A`, `B`, and `U` stand for `Married`, `Single` and `Unknown` respectively. Interestingly, this plot efficiently captures classical differences between married and single life, as we observe that **households with married members have more often lower income and must balance with lower expenses**, while it is completely opposite for **single member households, as they more frequently have higher income and are able to indulge in higher expenses**.  

#### Homeowner Type

In [ ]:
plot_demographic('HOMEOWNER_DESC')

Analyzing the distribution of the type of homeownership we again obtain expected results, as it is understandable that it is most probable for households who are able to afford their own place of residence to have higher income and expenses, while renters usually have better sense of utilizing their limited income. 

#### Household Composition

In [ ]:
plot_demographic('HH_COMP_DESC')

Among the different household compositions recorded in the data, we do not observe a lot of variety and this feature is not a good discriminator. However, we do observe two interesting discrepancies which can also be expected: couples with no children have the highest chance to be in the group with the highest income and expenses, while single parents tend to have a 'harder time'.

#### Household Size

In [ ]:
plot_demographic('HOUSEHOLD_SIZE_DESC')

The household size feature also displays very little significance for group distinction across different values. However, we can observe a trend that as the household size increases the probability of having lower income and expenses decreases, as only households with higher income can afford to have more children, but with that their expenses also increase.

#### Number of Children

In [ ]:
plot_demographic('KID_CATEGORY_DESC')

The demographic records for the number of children can be inferred from previous features such as household size and household composition and as such in the plot we observe a similar trend as in the previous analysis of household size.

## Part 4: Applied ML

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

# Map income categories to their geometric means of the bins
geo_means = pd.Series(data=geo_means, index=income_desc.index, name='INCOME_DESC_GEO_MEANS')
income_desc_geo_means = df_hh_demographic['INCOME_DESC'].apply(lambda x: geo_means[x]).astype('float')

# Calculate income / expenses ratios
feature_income_and_average_bill_amount = income_desc_geo_means / df_households_basket_payment_average
feature_income_and_average_median_product_price = income_desc_geo_means / df_households_basket_median_product_price_average
X = pd.concat((feature_income_and_average_bill_amount, feature_income_and_average_median_product_price), axis=1)

# Normalize the features to [0, 1]
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X= pd.DataFrame(X, columns=['Income / Average Bill Amount', 'Income / Average Median Product Price'])

# Cluster the 2D data using K-means
k = 3
kmeans = KMeans(n_clusters=k).fit(X)
labels = pd.Series(kmeans.labels_ + 1, name='Cluster')
X_labeled = pd.concat((X, labels), axis=1)

f, ax = plt.subplots(figsize=(14, 12))
sns.scatterplot(x='Income / Average Bill Amount', y='Income / Average Median Product Price', hue='Cluster', data=X_labeled, palette=sns.color_palette('muted', n_colors=k))
plt.show()

We are interested in whether there is existence of clusters in `hh_demographic` dataset. Since all the columns are categorical, we first transform the dataset using one-hot-encoding per column. Then, we apply PCA to project the dataset into 2 dimensional space, and finally apply KMeans clustering.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Do not include the households with 'unknown' values for some demographic features
c = (df_hh_demographic['MARITAL_STATUS_CODE'] != 'U') & \
    (df_hh_demographic['HH_COMP_DESC'] != 'Unknown') & \
    (df_hh_demographic['HOMEOWNER_DESC'] != 'Unknown') & \
    (df_hh_demographic['KID_CATEGORY_DESC'] != 'None/Unknown')

# Set number of clusters
k = 3

# One-hot encode categorical demographic features
X = pd.get_dummies(df_hh_demographic)

# Use PCA to project the demographic data into 2 dimensions
pca = PCA(n_components=2).fit(X)
X_proj = pca.transform(X)

# Cluster the 2D data using K-means
kmeans = KMeans(n_clusters=k).fit(X_proj)
labels = kmeans.labels_.reshape(-1, 1) + 1

df = pd.DataFrame(np.concatenate((X_proj, labels), axis=1), columns=['Principal Component 1', 'Principal Component 2', 'Cluster'])
df['Cluster'] = df['Cluster'].astype(int)

f, ax = plt.subplots(figsize=(14, 12))
sns.scatterplot(x='Principal Component 1', y='Principal Component 2',  data=df, hue='Cluster', palette=sns.color_palette('muted', n_colors=k))
plt.show()

Although, PCA is hard to interpret, however, in the graph above we can see that there are 2 or 3 distinguishable clusters. This might suggest of cluster existence in higher dimensions and we will use this information for further insights in the final version of the project. It is also evident that there are roughly 16 smaller cluster in the above plot. This is due to the discretization of the data. Even after performing PCA, the discrete values become part of the two main components, and therefore also discretize the data into groups. Therefore, we can safely ignore those 16 smaller groups, as they are no 'deeper' meaning.

In [ ]:
df = df_hh_demographic.copy()
df['Cluster'] = labels

print(df[df['Cluster'] == 1]['HOUSEHOLD_SIZE_DESC'].value_counts())
print()
print(df[df['Cluster'] == 2]['HOUSEHOLD_SIZE_DESC'].value_counts())
print()
print(df[df['Cluster'] == 3]['HOUSEHOLD_SIZE_DESC'].value_counts())

From the data above, we can see that after projecting the data in lower dimension, the clusters in higher dimensions are preserved. For example, taking the household size into account, the first cluster is consisted mainly of households with 2 adults and kinds, the second cluster is consisted of 2 adults with no kids, and the third cluster of singles.

## Part 5: Recap

In this milestone we managed to investigate the data, check for missing values and inconsistencies, do feature expansion, and identify the distribution of the most expressive columns. Furthermore, we analysed the household income categories, how much money households spend on average and by how much is that ammount defined by the products price. Additionaly, we saw the coupon usages among income categories, and looked for trends in daily expenses. Likewise, we investigated how the expenditure is distributed amongst goods and whether households prefer some products over others. Finaly, we did extensive demographic analysis and searched for clusters in the data.

## Part 6: Future Steps

#### Alternative to Average Bill Amount

As opposed to using the average bill amount, experiment with either number of items or average item price because:

1. The average number of items purchased is heavily related to how many family members there are (large family $\implies$ more stuff to buy)
2. The average item price is more related to how wealthily a person is (more money $\implies$ more fancy organic products).

#### Alternative to Average Price of the Most Expensive Product

The more items you buy, the more likely you are to buy something expensive. Further investigate the average item price.

#### In Depth Analysis of Daily Expenditure

We saw that households do not shop every day equally. We would like to see why is that the case.

#### Outliers

There are plots with significant outliers. We will look at those in order to find out what they are and if they influence the dataset in any statistically significant way. Maybe also compare how the average bill amount changes with the demographic data.

#### Use the Clusters

Use the clusters data obtained with PCA to further analyse the expenditure.